##### Programming Exercise by Jomer Allan G. Barcenilla Based on:
UNDERSTANDING DEEP LEARNING REQUIRES RETHINKING GENERALIZATION
- Chiyuan Zhang, Samy Bengio, Moritz Hardt, Benjamin Recht, & Oriol Vinyals

### Import Libraries

In [1]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

### Create the Inception Model

In [2]:
def conv_module(x, filters, kernel_size, strides):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def inception_module(x, ch1, ch3):
    conv1 = conv_module(x, ch1, (1, 1), 1)
    conv2 = conv_module(x, ch3, (3, 3), 1)
    inception = Concatenate(axis=-1)([conv1, conv2])
    return inception

def downsample_module(x, ch3):
    conv1 = conv_module(x, ch3, (3, 3), 2)
    conv2 = conv_module(x, ch3, (3, 3), 2)
    x = Concatenate(axis=-1)([conv1, conv2])
    return x

def build_model():
    inputs = Input(shape=(28, 28, 3))
    x = inputs

    # Inception (Small)
    x = conv_module(x, 96, (3, 3), 1)
    x = inception_module(x, 32, 32)
    x = inception_module(x, 32, 48)
    x = downsample_module(x, 80)    
    x = inception_module(x, 112, 48)
    x = inception_module(x, 96, 64)
    x = inception_module(x, 80, 80)
    x = inception_module(x, 48, 96)
    x = downsample_module(x, 96)
    x = inception_module(x, 176, 160)
    x = inception_module(x, 176, 160)

    # Global Average Pooling
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)

    # Fully Connected layer
    outputs = Dense(10, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

model = build_model()

In [3]:
# Show model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 28, 28, 96)           2688      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 28, 28, 96)           384       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 28, 28, 96)           0         ['batch_normalization[0][0